In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot, stats, holostats
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import plot
import flux
import ionfrac

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D
from tqdm import tqdm


In [ ]:
# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz
scales = np.array([500, 1000, 2000, 4000, 8000, 16000, 25000])
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array([4000, 9000, 21000, 39000, 54000, 68000, 94000])
masss = np.array([1.4, 2, 4, 8, 12, 16, 24])


file = np.load(VICO_loc+'/AngladaRosero2019/AngladaData/shock_flux_arrays.npz')
lums = file['luminosities']
file.close()

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp=False)
    snaps[ii].load_shock_variables()

# Case A

## calculate avg maps arrays

In [ ]:
for snap in tqdm(snaps):
    ionfrac.calculate_column_mass_densities(snap)

# load and print

In [ ]:
# nu = 5.3*10**9
# for scale in [4000, 25000]:
#     for ss, snap in enumerate(tqdm(snaps)):
#         string = ''
#         if ss==0: string += str(scale)
#         string += f"\t&\t{masss[ss]}"
#         for v_cutoff in [1, 10, 100]:
#             string += f"\t&\t{ionfrac.mass_avg_3d_ionfrac(snap, v_cutoff, scale):.4f}"
#             string += f"\t&\t{ionfrac.vol_avg_3d_ionfrac(snap, v_cutoff, scale):.4f}"
#             string += f"\t&\t{ionfrac.emis_avg_3d_ionfrac(snap, nu, v_cutoff, scale):.4f}"
#         string += '\\'
#         print(string)
#     if scale == 4000: print("\\hline")

In [ ]:
for snap in tqdm(snaps):
    ionfrac.load_average_ionfrac_array(snap)

In [ ]:
nu = 5.3*10**9
ff = 5
for scale in [4000, 25000]:
    for ss, snap in enumerate(snaps):
        string = ''
        if ss==0: string += str(scale)
        string += f"\t&\t{masss[ss]}"
        for v_cutoff in [0,1,2]:
            string += f"\t&\t{ionfrac.mass_avg_map_ionfrac(snap, v_cutoff, scale):.4f}"
            string += f"\t&\t{ionfrac.vol_avg_map_ionfrac(snap, v_cutoff, scale):.4f}"
            string += f"\t&\t{ionfrac.emis_avg_map_ionfrac(snap, ff, v_cutoff, scale):.4f}"
        string += '\\\\'
        print(string)
    if scale == 4000: print("\\hline")

# Case B

In [ ]:
for ss in range(0,len(snaps)):
    ionfrac.load_average_ionfrac_array(snaps[ss])
    ionfrac.load_average_ionfrac_array_ratio(snaps[ss], const=False)
    print(stats(snaps[ss].ionfrac_emis_ratio))

In [ ]:
for snap in tqdm(snaps):
    snap.load_intensity_variables_const(5.3*10**9, const=False, r_kpc=1)
    ionfrac.load_average_ionfrac_array_ratio(snap, debug=True)
cooled = True

In [ ]:
print(snaps[0].__dict__.keys())

## Case B Map Avg Ionfrac

In [ ]:
cooled=True
nu = 5.3*10**9
ff = 5
for scale in [4000, 25000]:
    for ss, snap in enumerate(snaps):
        string = ''
        if ss==0: string += str(scale)
        string += f"\t&\t{masss[ss]}"
        for v_cutoff in [0,1,2]:
            string += f"\t&\t{ionfrac.mass_avg_map_ionfrac(snap, v_cutoff, scale, cooled=cooled):.4f}"
            string += f"\t&\t{ionfrac.vol_avg_map_ionfrac(snap, v_cutoff, scale, cooled=cooled):.4f}"
            string += f"\t&\t{ionfrac.emis_avg_map_ionfrac(snap, ff, v_cutoff, scale, cooled=cooled):.4f}"
        string += '\\\\'
        print(string)
    if scale == 4000: print("\\hline")

## Case B 3D avg ionfrac

In [ ]:
# cooled=True
# nu = 5.3*10**9
# ff = 5
# for scale in [4000, 25000]:
#     for ss, snap in enumerate(snaps):
#         string = ''
#         if ss==0: string += str(scale)
#         string += f"\t&\t{masss[ss]}"
#         for v_cutoff in [0,1,2]:
#             string += f"\t&\t{ionfrac.mass_avg_3d_ionfrac(snap, v_cutoff, scale, cooled=cooled):.4f}"
#             string += f"\t&\t{ionfrac.vol_avg_3d_ionfrac(snap, v_cutoff, scale, cooled=cooled):.4f}"
#             string += f"\t&\t{ionfrac.emis_avg_3d_ionfrac(snap, ff, v_cutoff, scale, cooled=cooled):.4f}"
#         string += '\\\\'
#         print(string)
#     if scale == 4000: print("\\hline")

In [ ]:
np.set_printoptions(precision=4)
for snap in snaps:
    print(snap.name)
    print('mass:', np.percentile(snap.ionfrac_mass_ratio[0], (0,25,50,75,100)))
    print('vol:', np.percentile(snap.ionfrac_vol_ratio[0], (0,25,50,75,100)))
    print('emis:', np.percentile(snap.ionfrac_emis_ratio[5,0], (0,25,50,75,100)))

In [ ]:
print(np.percentile(snaps[3].ionfrac_emis_ratio, (5,10,15,50,75,100)))

In [ ]:
print(snaps[0].ionfrac_emis_ratio.shape)

In [ ]:
npfile = np.load('/Users/emigardiner/VICO/pjams-ionization/Data/Snap039_n/Snap039_n_average_ionfrac_arrays.npz')
print(npfile.files)

In [ ]:
for snap in snaps:
    ionfrac.load_average_ionfrac_array_ratio(snap)
    print(np.percentile(snap.ionfrac_emis_ratio[5], (0,25,50,75,98,99,100)))

In [ ]:
def mass_ionfrac_array_const(Snap, v_mins=[1,10,100], const=False):
    ionfrac_mass = np.zeros((len(v_mins),len(Snap.x1), len(Snap.x2)))
    for v in range(len(v_mins)):
        for i in range(len(Snap.x1)):
            for j in range(len(Snap.x2)):
                numer = 0
                denom = 0
                for k in range(len(Snap.x3)):
                    if(Snap.q['v1'][i,j,k] >= v_mins[v]):
                        if(const): scale_factor = const
                        else: 
                            scale_factor = Snap.cooling_times[i,j,k]/Snap.flow_times[i,j,k]
                            if(scale_factor>1): scale_factor=1 # can't be more than 1
                        numer += (Snap.ion_fractions[i,j,k] * Snap.q['d'][i,j,k] 
                                  * Snap.del3[k] * scale_factor)
                        denom += Snap.q['d'][i,j,k] *  Snap.del3[k]
                if(denom>0): ionfrac_mass[v,i,j] = numer/denom
                else: ionfrac_mass[v,i,j] = 0  
    if(const): Snap.ionfrac_mass_const = ionfrac_mass
    else: Snap.ionfrac_mass_ratio = ionfrac_mass

# Calculating Ionization Fraction Arrays
def vol_ionfrac_array_const(Snap, v_mins=[1,10,100], const=False):
    ionfrac_vol = np.zeros((len(v_mins),len(Snap.x1), len(Snap.x2)))
    for v in range(len(v_mins)):
        for i in range(len(Snap.x1)):
            for j in range(len(Snap.x2)):
                numer = 0
                denom = 0
                for k in range(len(Snap.x3)):
                    if(Snap.q['v1'][i,j,k] >= v_mins[v]):
                        if(const): scale_factor = const
                        else: 
                            scale_factor = Snap.cooling_times[i,j,k]/Snap.flow_times[i,j,k]
                            if(scale_factor>1): scale_factor=1 # can't be more than 1
                        numer += Snap.ion_fractions[i,j,k] * Snap.del3[k] * scale_factor
                        denom += Snap.del3[k]
                if(denom>0): ionfrac_vol[v,i,j] = numer/denom
                else: ionfrac_vol[v,i,j] = 0  
    if(const): Snap.ionfrac_vol_const = ionfrac_vol
    else: Snap.ionfrac_vol_ratio = ionfrac_vol

def emis_ionfrac_array_const(Snap, v_mins=[1,10,100], f=5, const=False):
    Snap.load_intensity_variables(FREQS[f])
    ionfrac_emis = np.zeros((len(FREQS),len(v_mins),len(Snap.x1), len(Snap.x2)))
    for v in range(len(v_mins)):
        for i in range(len(Snap.x1)):
            for j in range(len(Snap.x2)):
                numer = 0
                denom = 0
                for k in range(len(Snap.x3)):
                    if(Snap.q['v1'][i,j,k] >= v_mins[v]):
                        if(const): scale_factor = const
                        else: 
                            scale_factor = Snap.cooling_times[i,j,k]/Snap.flow_times[i,j,k]
                            if(scale_factor>1): scale_factor=1 # can't be more than 1
                        numer += Snap.ion_fractions[i,j,k] * Snap.emission_coefs[i,j,k] * Snap.del3[k] * scale_factor
                        denom += Snap.emission_coefs[i,j,k] * Snap.del3[k] 
                if(denom>0): ionfrac_emis[f,v,i,j] = numer/denom
                else: ionfrac_emis[f,v,i,j] = 0 
    if(const): Snap.ionfrac_emis_const = ionfrac_emis
    else: Snap.ionfrac_emis_ratio = ionfrac_emis

In [ ]:
cooled=True
nu = 5.3*10**9
ff = 5
for scale in [4000, 25000]:
    for ss, snap in enumerate((snaps)):
        mass_ionfrac_array_const(snap)
        vol_ionfrac_array_const(snap)
        emis_ionfrac_array_const(snap)
        string = ''
        if ss==0: string += str(scale)
        string += f"\t&\t{masss[ss]}"
        for v_cutoff in [0,1,2]:
            string += f"\t&\t{ionfrac.mass_avg_map_ionfrac(snap, v_cutoff, scale, cooled=cooled):.4f}"
            string += f"\t&\t{ionfrac.vol_avg_map_ionfrac(snap, v_cutoff, scale, cooled=cooled):.4f}"
            string += f"\t&\t{ionfrac.emis_avg_map_ionfrac(snap, ff, v_cutoff, scale, cooled=cooled):.4f}"
        string += ' \\\\'
        print(string)
    if scale == 4000: print("\\hline")

In [ ]:
np.set_printoptions(precision=4)
# for scale in [4000, 25000]:
for ss, snap in enumerate((snaps)):
    print(snap.name)
    emis_ionfrac_array_const(snap)
    print(f"new: {holostats(snap.ionfrac_emis_ratio)}")
    ionfrac.load_average_ionfrac_array_ratio(snap)
    print(f"old: {holostats(snap.ionfrac_emis_ratio)}")

# Fix bad ionfracs

In [ ]:
for snap in snaps[[0,4,5,6]]:
    mass_ionfrac_array_const(snap)
    vol_ionfrac_array_const(snap)
    emis_ionfrac_array_const(snap)
    data_path = (VICO_loc+'/Data/'+snap.name+'/')
    savename = data_path+snap.name+'_average_ionfrac_arrays_const'+str(False)+'.npz'
    np.savez(savename, 
        ionfrac_mass = snap.ionfrac_mass_ratio, 
        ionfrac_vol = snap.ionfrac_vol_ratio,
        ionfrac_emis = snap.ionfrac_emis_ratio
    )
